## Memory-Powered Agent
Memory-powered agents are advanced AI systems that incorporate long-term memory to improve their reasoning, adaptability, and contextual understanding. Unlike traditional models that rely solely on short-term prompts and retrieval mechanisms, memory-powered agents store and recall relevant past interactions, facts, or experiences. This enables them to provide more coherent, contextually aware, and personalized responses over time.

These agents utilize structured and unstructured memory storage, often integrating vector databases or specialized memory architectures like recurrent models, attention mechanisms, or embeddings. By dynamically retrieving and synthesizing information from both immediate context and past experiences, they enhance decision-making, reduce redundant queries, and maintain continuity in conversations or tasks.

A key advantage of memory-powered agents is their ability to learn and refine their responses over time. They can adapt to user preferences, remember previous conversations, and even correct past mistakes, making them highly effective in applications like customer support, virtual assistants, and AI-driven research tools. Combining retrieval-based and generative approaches, these agents push the boundaries of intelligent automation and user experience.

#### 1. Importing Dependencies


In [1]:
import mcp
import os
import json
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field

#### 2. Memory-Powered Conversational Agent

This code defines four Pydantic models that serve as input schemas for Model Context Protocol (MCP) tools in your memory-powered agent:

1. ChatInput : This model captures user messages for the chat functionality.
   
   - It has a single required field message which stores the text input from the user.
   - The description parameter provides documentation for the field.

2. SummaryInput : This model configures how conversation summaries are generated.
   
   - It has a boolean field detailed that defaults to False .
   - When detailed is True , the agent will generate a more comprehensive summary.
   - When False (default), it will create a concise summary.

3. SaveInput : This model specifies where to save the conversation history.
   
   - It has a filename field with a default value of "memory.json".
   - This allows users to either use the default filename or specify a custom one.

4. LoadInput : This model specifies which saved conversation to load.
   
   - Similar to SaveInput , it has a filename field defaulting to "memory.json".
   - This allows users to load conversation history from a specified file.
   
These Pydantic models provide several benefits:

- Type validation for inputs
- Default values where appropriate
- Self-documentation through descriptions
- Structured interface for the MCP tools
The models will be used with the @mcp.tool() decorated methods in your agent class to provide a standardized way to interact with the agent's functionality.

In [2]:
# Define input schemas for MCP tools
class ChatInput(BaseModel):
    message: str = Field(..., description="User message to process")

class SummaryInput(BaseModel):
    detailed: bool = Field(False, description="Whether to generate a detailed summary")

class SaveInput(BaseModel):
    filename: str = Field("memory.json", description="Filename to save memory to")

class LoadInput(BaseModel):
    filename: str = Field("memory.json", description="Filename to load memory from")

#### 3. Example Usage of Memory-Powered Agent

This code implements a memory-powered conversational agent that uses the Model Context Protocol (MCP) to expose its functionality as tools. The agent maintains conversation history and provides several key capabilities through MCP-decorated methods:

1. Chat functionality for processing user messages and generating contextual responses
2. Conversation summarization with adjustable detail levels
3. Persistence through save and load operations
4. OpenAI integration using the latest API format
The implementation includes fallback compatibility for environments where MCP might not be fully available, ensuring the agent can work in various contexts while maintaining its core memory capabilities.

In [3]:
# Create a compatible tool decorator if it doesn't exist
if not hasattr(mcp, 'tool'):
    mcp.tool = lambda name: lambda func: func

# Memory-powered agent with MCP tools
class MemoryPoweredAgent:
    def __init__(self, api_key=None):
        if api_key:
            os.environ["OPENAI_API_KEY"] = api_key
        self.messages = []
        self.init_messages()
    
    def init_messages(self):
        # Initialize with system message
        self.messages = [
            {"role": "system", "content": "You are a helpful assistant with memory of the conversation."}
        ]
    
    @mcp.tool("chat")
    def chat(self, input_data: ChatInput) -> Dict[str, Any]:
        """Process a user message and generate a response"""
        # Add user message to history
        self.messages.append({"role": "user", "content": input_data.message})
        
        # Get response from OpenAI
        response = self._get_completion()
        
        # Add assistant response to history
        self.messages.append({"role": "assistant", "content": response})
        
        return {
            "response": response,
            "message_count": len(self.messages) - 1  # Exclude system message
        }
    
    @mcp.tool("summarize")
    def summarize(self, input_data: SummaryInput = SummaryInput()) -> Dict[str, Any]:
        """Generate a summary of the conversation"""
        if len(self.messages) <= 1:
            return {"summary": "No conversation to summarize yet."}
        
        # Create a prompt for summarization
        detail_level = "detailed" if input_data.detailed else "concise"
        summary_prompt = f"Please provide a {detail_level} summary of our conversation so far."
        
        # Add temporary message for summary request
        self.messages.append({"role": "user", "content": summary_prompt})
        
        # Get summary from OpenAI
        summary = self._get_completion()
        
        # Remove the temporary message
        self.messages.pop()
        
        return {"summary": summary}
    
    @mcp.tool("save")
    def save(self, input_data: SaveInput) -> Dict[str, Any]:
        """Save conversation memory to a file"""
        filename = input_data.filename
        with open(filename, 'w') as f:
            json.dump(self.messages, f)
        return {"status": "success", "filename": filename}
    
    @mcp.tool("load")
    def load(self, input_data: LoadInput) -> Dict[str, Any]:
        """Load conversation memory from a file"""
        filename = input_data.filename
        try:
            with open(filename, 'r') as f:
                self.messages = json.load(f)
            return {"status": "success", "filename": filename, "message_count": len(self.messages) - 1}
        except FileNotFoundError:
            return {"status": "error", "message": f"File {filename} not found"}
    
    def _get_completion(self):
        """Get completion from OpenAI API"""
        import openai
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0.7
        )
        return response.choices[0].message.content

#### 4. Interactive Demo for Memory-Powered Agent
This function provides a command-line interface to interact with the MemoryPoweredAgent. Users can chat with the agent, request conversation summaries, save and load memory, or exit the session. The agent remembers previous interactions, enabling context-aware responses for a more natural conversation experience.

The demo supports:

- Live chatting with memory retention
- Summarization of past interactions
- Saving and loading memory for persistent context
- Exit functionality to end the session

In [4]:
# Example usage
if __name__ == "__main__":
    # Create agent
    agent = MemoryPoweredAgent()
    
    # Example of direct usage
    response = agent.chat(ChatInput(message="Hello, how are you today?"))
    print(f"Agent: {response['response']}")
    
    # Example of using MCP
    try:
        # Try to use MCP to execute the tool
        result = mcp.execute("chat", {"message": "Tell me about yourself"})
        print(f"MCP Agent: {result['response']}")
    except Exception as e:
        # Fall back to direct execution if MCP fails
        response = agent.chat(ChatInput(message="Tell me about yourself"))
        print(f"Agent: {response['response']}")
    
    # Simple command-line interface
    print("\nMemory Agent ready. Type 'exit' to quit, 'summary' for conversation summary.")
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'summary':
            print("\n===== CONVERSATION SUMMARY =====")
            summary_result = agent.summarize()
            print(summary_result["summary"])
            print("================================\n")
        elif user_input.lower() == 'save':
            filename = input("Filename to save (default: memory.json): ") or "memory.json"
            save_result = agent.save(SaveInput(filename=filename))
            print(f"Saved to {save_result['filename']}")
        elif user_input.lower() == 'load':
            filename = input("Filename to load (default: memory.json): ") or "memory.json"
            load_result = agent.load(LoadInput(filename=filename))
            if load_result["status"] == "success":
                print(f"Loaded from {load_result['filename']}")
            else:
                print(load_result["message"])
        else:
            chat_result = agent.chat(ChatInput(message=user_input))
            print(f"\nAgent: {chat_result['response']}")

Agent: Hello! I'm here and ready to assist you. How can I help you today?
Agent: I'm a helpful assistant designed to provide information and support. I can assist you with a wide range of topics and questions. Just let me know how I can help you!

Memory Agent ready. Type 'exit' to quit, 'summary' for conversation summary.

Agent: Hello, Alexandra! It's nice to meet you. How can I assist you today?

Agent: That's wonderful! Cats are amazing animals. Is there anything specific you'd like to know or discuss about cats?

Agent: Of course! Feel free to let me know what you'd like to talk about or ask about, and I'll do my best to assist you.

Agent: You mentioned earlier that you love cats. Is there anything else you love or would like to talk about? Feel free to share your interests or ask any questions you have in mind.

Agent: If you have any questions or need assistance with anything, feel free to ask. I'm here to help!

Agent: If you need any assistance in the future, don't hesitate t

KeyboardInterrupt: 